# Load Libraries

In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Set Path

In [2]:
data_dir = Path("../input/google-smartphone-decimeter-challenge")

# Load Data

In [3]:
df_train = pd.read_pickle(str(data_dir / "gsdc_cleaned_train.pkl.gzip"))

In [4]:
df_test = pd.read_pickle(str(data_dir / "gsdc_cleaned_test.pkl.gzip"))

In [5]:
for col in df_train.columns:
    print(col)

collectionName
phoneName
millisSinceGpsEpoch
latDeg
lngDeg
heightAboveWgs84EllipsoidM
phone
timeSinceFirstFixSeconds
hDop
vDop
speedMps
courseDegree
t_latDeg
t_lngDeg
t_heightAboveWgs84EllipsoidM
constellationType
svid
signalType
receivedSvTimeInGpsNanos
xSatPosM
ySatPosM
zSatPosM
xSatVelMps
ySatVelMps
zSatVelMps
satClkBiasM
satClkDriftMps
rawPrM
rawPrUncM
isrbM
ionoDelayM
tropoDelayM
utcTimeMillis
TimeNanos
LeapSecond
FullBiasNanos
BiasNanos
BiasUncertaintyNanos
DriftNanosPerSecond
DriftUncertaintyNanosPerSecond
HardwareClockDiscontinuityCount
Svid
TimeOffsetNanos
State
ReceivedSvTimeNanos
ReceivedSvTimeUncertaintyNanos
Cn0DbHz
PseudorangeRateMetersPerSecond
PseudorangeRateUncertaintyMetersPerSecond
AccumulatedDeltaRangeState
AccumulatedDeltaRangeMeters
AccumulatedDeltaRangeUncertaintyMeters
CarrierFrequencyHz
MultipathIndicator
ConstellationType
AgcDb
BasebandCn0DbHz
FullInterSignalBiasNanos
FullInterSignalBiasUncertaintyNanos
SatelliteInterSignalBiasNanos
SatelliteInterSignalBiasUnc

# Extract Data

## Get dlatDeg, dlngDeg, dheightAboveWgs84EllipsoidM

In [6]:
def getDeltaPositions(df:pd.DataFrame)->pd.DataFrame:
    output = df.copy()
    phones = output['phone'].unique()
    output[['dlatDeg', 'dlngDeg', 'dheight']] = 0
    for phone in phones:
        latDeg = output.loc[output['phone'] == phone, 'latDeg']
        output.loc[output['phone'] == phone, 'dlatDeg'] = latDeg - latDeg.iloc[0]
        
        lngDeg = output.loc[output['phone'] == phone, 'lngDeg']
        output.loc[output['phone'] == phone, 'dlngDeg'] = lngDeg - lngDeg.iloc[0]
        
        heightAboveWgs84EllipsoidM = output.loc[output['phone'] == phone, 'heightAboveWgs84EllipsoidM']
        output.loc[output['phone'] == phone, 'dheight'] = heightAboveWgs84EllipsoidM - heightAboveWgs84EllipsoidM.iloc[0]
        
    return output

In [7]:
df_train = getDeltaPositions(df_train)
df_test = getDeltaPositions(df_test)

## Scaler

In [8]:
class Scaler():
    def __init__(self, target_columns):
        self.target_columns = target_columns
        self.renames_columns = [name + "_Scaled" for name in target_columns]
        self.scaler = dict()
        for name in target_columns:
            self.scaler[name] = StandardScaler()
            
    def fit(self, df:pd.DataFrame):
        for col in self.target_columns:
            self.scaler[col].fit(df[col].values.reshape(-1, 1))
            
    def transform(self, df:pd.DataFrame):
        output = df.copy()
        for col,recol in zip(self.target_columns, self.renames_columns):
            output[recol] = self.scaler[col].transform(df[col].values.reshape(-1, 1))
            output[recol].fillna(0, inplace = True)
        return output
        
    

In [9]:
target_columns = ['dlatDeg', 'dlngDeg', 'dheight',
                 'xSatPosM', 'ySatPosM', 'zSatPosM', 'xSatVelMps', 'ySatVelMps', 'zSatVelMps',
                  'UncalGyroXRadPerSec', 'UncalGyroYRadPerSec', 'UncalGyroZRadPerSec', 
                  'DriftXRadPerSec', 'DriftYRadPerSec', 'DriftZRadPerSec', 
                  'UncalAccelXMps2', 'UncalAccelYMps2', 'UncalAccelZMps2', 'BiasXMps2', 'BiasYMps2', 'BiasZMps2',
                  'UncalMagXMicroT', 'UncalMagYMicroT', 'UncalMagZMicroT', 'BiasXMicroT', 'BiasYMicroT', 'BiasZMicroT',
                  'yawDeg', 'rollDeg', 'pitchDeg'
                 ]

In [10]:
scaler = Scaler(target_columns)
scaler.fit(df_train)

df_train = scaler.transform(df_train)
df_test = scaler.transform(df_test)

In [11]:
df_train[scaler.renames_columns].describe()

,dlatDeg_Scaled,dlngDeg_Scaled,dheight_Scaled,xSatPosM_Scaled,ySatPosM_Scaled,zSatPosM_Scaled,xSatVelMps_Scaled,ySatVelMps_Scaled,zSatVelMps_Scaled,UncalGyroXRadPerSec_Scaled,...,BiasZMps2_Scaled,UncalMagXMicroT_Scaled,UncalMagYMicroT_Scaled,UncalMagZMicroT_Scaled,BiasXMicroT_Scaled,BiasYMicroT_Scaled,BiasZMicroT_Scaled,yawDeg_Scaled,rollDeg_Scaled,pitchDeg_Scaled
count,1.313420e+05,1.313420e+05,1.313420e+05,1.313420e+05,1.313420e+05,1.313420e+05,1.313420e+05,1.313420e+05,1.313420e+05,1.313420e+05,...,1.313420e+05,1.313420e+05,1.313420e+05,1.313420e+05,1.313420e+05,1.313420e+05,1.313420e+05,1.313420e+05,1.313420e+05,1.313420e+05
mean,5.390932e-17,-7.898406e-18,5.193472e-17,-3.786907e-17,-8.926280e-17,-1.276729e-17,7.487256e-17,1.244269e-17,1.890748e-17,-4.091212e-18,...,-8.461032e-17,1.765240e-16,-8.222998e-18,1.493123e-17,-8.114800e-18,-6.491840e-19,3.949203e-18,-4.744453e-17,-1.666239e-17,2.185586e-17
std,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,9.308231e-01,...,7.542804e-01,9.307495e-01,9.307495e-01,9.306759e-01,7.541188e-01,7.541188e-01,7.540280e-01,5.278070e-01,5.276843e-01,5.276843e-01
min,-3.367212e+00,-2.837517e+00,-8.485201e+01,-1.824201e+00,-1.533469e+00,-2.896326e+00,-2.966502e+00,-2.135619e+00,-1.592272e+00,-2.147876e+01,...,-2.065078e+00,-3.683841e+00,-3.661762e+00,-3.285377e+00,-3.146779e+00,-1.970667e+00,-2.886818e+00,-1.577129e+00,-1.465147e+00,-7.969673e-01
25%,-1.426406e-01,-2.570525e-01,-3.623789e-01,-7.641287e-01,-7.586592e-01,-7.307094e-01,-6.466295e-01,-7.540929e-01,-1.008162e+00,-2.254101e-01,...,0.000000e+00,-4.797574e-01,-5.900799e-02,-3.370902e-01,-2.461699e-01,0.000000e+00,-3.088827e-01,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.784625e-02,8.844239e-02,-2.357607e-01,-2.086152e-01,-1.548048e-01,3.174404e-01,-1.229942e-01,5.439754e-02,-1.034868e-03,0.000000e+00,...,0.000000e+00,0.000000e+00,1.489335e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.870814e-01,2.033670e-01,2.792265e-01,7.479192e-01,5.701537e-01,8.245646e-01,7.530651e-01,6.654773e-01,9.952601e-01,2.263081e-01,...,4.842431e-01,4.070920e-01,4.331973e-01,1.884584e-01,8.105131e-02,1.906092e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.689582e+00,2.871625e+00,1.882394e+02,2.505145e+00,3.481311e+00,2.111477e+00,2.183880e+00,2.622603e+00,1.575673e+00,2.081124e+01,...,4.842431e-01,3.249819e+00,4.818309e+00,3.778347e+00,2.810664e+00,3.914161e+00,2.720061e+00,2.416760e+00,8.816943e-01,2.381081e+00


#  Output

In [12]:
df_train.to_pickle(str(data_dir / "gsdc_extract_train.pkl.gzip"))
df_test.to_pickle(str(data_dir / "gsdc_extract_test.pkl.gzip"))